This notebook was run on Google Colab

### Mount Main Google Drive

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


### GDrive Mounting (PyDrive2)

In [5]:
# !pip install PyDrive2

In [6]:
# from pydrive2.auth import GoogleAuth
# from pydrive2.drive import GoogleDrive
# from google.colab import auth
# from oauth2client.client import GoogleCredentials

# # Authenticate and create the PyDrive client for acc2
# auth.authenticate_user()
# gauth = GoogleAuth()
# gauth.credentials = GoogleCredentials.get_application_default()
# drive2 = GoogleDrive(gauth)

In [7]:
# # List files in a specific folder in the second Google Drive
# folder_id = '1-K79qAXkFCL0KUhztx9YOWiLz5iNGOtk'  # Replace with the actual folder ID
# file_list = drive2.ListFile({'q': f"'{folder_id}' in parents and trashed=false"}).GetList()

# for file in file_list:
#     print(f"title: {file['title']}, id: {file['id']}")

### GDrive Mounting (GCP)

In [8]:
# !pip install google-auth google-auth-oauthlib google-auth-httplib2 google-api-python-client

In [9]:
# # Mounting other google drives and getting into the files seems to connect with GCP, which for now is not yet explored.

# import os
# import google.auth
# from google.auth.transport.requests import Request
# from google_auth_oauthlib.flow import InstalledAppFlow
# from googleapiclient.discovery import build

# # Path to the uploaded credentials file
# creds_path = '/content/credentials.json'

# # Define the scopes
# SCOPES = ['https://www.googleapis.com/auth/drive']

# # Authenticate and create the service
# def authenticate_gdrive(creds_path):
#     creds = None
#     if os.path.exists('token.json'):
#         creds = google.auth.load_credentials_from_file('token.json', SCOPES)
#     if not creds or not creds.valid:
#         if creds and creds.expired and creds.refresh_token:
#             creds.refresh(Request())
#         else:
#             flow = InstalledAppFlow.from_client_secrets_file(creds_path, SCOPES)
#             creds = flow.run_local_server(port=0)
#         with open('token.json', 'w') as token:
#             token.write(creds.to_json())
#     return build('drive', 'v3', credentials=creds)

# # Authenticate and create the Drive service
# service = authenticate_gdrive(creds_path)

# # Function to list files in a specific folder
# def list_files_in_folder(service, folder_id):
#     query = f"'{folder_id}' in parents"
#     results = service.files().list(q=query, pageSize=10).execute()
#     items = results.get('files', [])

#     if not items:
#         print('No files found.')
#     else:
#         print('Files:')
#         for item in items:
#             print(f"{item['name']} ({item['id']})")

# # Replace with the actual folder ID of 'Colab Notebooks/FINA4359'
# folder_id = 'your_folder_id_here'

# # List files in the specified folder
# list_files_in_folder(service, folder_id)

### Download RAPIDSAI CUML

In [10]:
import locale
locale.getpreferredencoding = lambda: "UTF-8"

!git clone https://github.com/rapidsai/rapidsai-csp-utils.git
!python rapidsai-csp-utils/colab/pip-install.py  # for this to run, please enable GPU runtime

Cloning into 'rapidsai-csp-utils'...
remote: Enumerating objects: 553, done.
remote: Counting objects: 100% (284/284), done.
remote: Compressing objects: 100% (182/182), done.
remote: Total 553 (delta 179), reused 147 (delta 100), pack-reused 269 (from 1)
Receiving objects: 100% (553/553), 178.44 KiB | 1.65 MiB/s, done.
Resolving deltas: 100% (281/281), done.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.5/40.5 kB 3.0 MB/s eta 0:00:00
Installing RAPIDS remaining 24.10.* libraries
Looking in indexes: https://pypi.org/simple, https://pypi.nvidia.com
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 567.7/567.7 MB 1.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 GB 914.8 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.3/4.3 MB 92.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 915.5/915.5 kB 54.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 83.6/83.6 kB 7.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [11]:
# !pip install gensim==4.1.0

In [12]:
import os
os.environ['CUDA_LAUNCH_BLOCKING'] = '1'

### Define Machine

#### Old

In [13]:
# # uses gensim (CPU based) for topic modeling

# import pandas as pd
# from cuml.feature_extraction.text import TfidfVectorizer
# from transformers import pipeline
# from sentence_transformers import SentenceTransformer, util
# import cupy as cp
# import numpy as np
# import re
# import time
# import torch
# from tqdm import tqdm
# from gensim.models.ldamulticore import LdaMulticore
# from gensim.corpora import Dictionary

# class ReviewAnalyzer:
#     def __init__(self, df, metrics, subset_indices=None, model_name='cardiffnlp/twitter-roberta-base-sentiment'):
#         if subset_indices is not None:
#             self.df = df.iloc[subset_indices]
#         else:
#             self.df = df
#         # df.reset_index(inplace=True)

#         self.reviews = self.df['Combined_Reviews']  # Assuming the review text is in a column named 'Combined_Reviews'
#         self.metrics = metrics
#         self.vectorizer = TfidfVectorizer(stop_words='english')

#         # This is for one GPU usage
#         # self.sentiment_pipeline = pipeline("sentiment-analysis", model=model_name, device=0)
#         # self.similarity_model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2', device='cuda')

#         # This if for multi GPU usage
#         # Disable parallelism before tokenization
#         os.environ["TOKENIZERS_PARALLELISM"] = "false"
#         self.sentiment_pipeline = pipeline("sentiment-analysis", model=model_name, device_map='auto', batch_size=128)
#         self.similarity_model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')
#         # Enable parallelism before tokenization
#         os.environ["TOKENIZERS_PARALLELISM"] = "true"
#         self.model_name = model_name

#     def preprocess(self):
#         self.X = self.vectorizer.fit_transform(self.reviews)
#         self.X = cp.array(self.X.toarray(), dtype=cp.float32)  # Convert to dense CuPy array

#     def get_topics(self):
#         texts = [re.split(r'\W+', review.lower()) for review in self.reviews]
#         dictionary = Dictionary(texts)
#         corpus = [dictionary.doc2bow(text) for text in texts]

#         # Initialize and train the Gensim LDA model with multicore support
#         lda_model = LdaMulticore(corpus, num_topics=len(self.metrics), id2word=dictionary, passes=10, workers=4, random_state=42)

#         topics = {}
#         for idx, topic in lda_model.print_topics(-1):
#             topics[idx] = topic

#         return topics

#     def analyze_sentiments(self):
#         segment_sentiments = []
#         for review in tqdm(self.reviews, desc="Processing Reviews for Sentiment Analysis"):
#             sentence_segments = re.split(r'(?<=\.)\s+(?=[A-Z])', review)
#             for segment in sentence_segments:
#                 if segment.strip():  # Ignore empty segments
#                     try:
#                         sentiment = self.sentiment_pipeline(segment.strip())[0]
#                         sentiment_score = -sentiment['score'] if sentiment['label'] == 'LABEL_0' else sentiment['score']
#                         segment_sentiments.append((segment.strip(), sentiment['label'], sentiment_score))
#                     except Exception as e:
#                         pass
#                         # print(f"Error processing segment: {segment.strip()}")
#                         # print(f"Error message: {e}")

#         return segment_sentiments

#     def analyze_topics(self):
#         segment_topics = []
#         texts = [re.split(r'\W+', review.lower()) for review in self.reviews]
#         dictionary = Dictionary(texts)
#         corpus = [dictionary.doc2bow(text) for text in texts]

#         # Initialize and train the Gensim LDA model with multicore support
#         lda_model = LdaMulticore(corpus, num_topics=len(self.metrics), id2word=dictionary, passes=10, workers=4, random_state=42)

#         for review in tqdm(self.reviews, desc="Processing Reviews for Topic Modeling"):
#             sentence_segments = re.split(r'(?<=\.)\s+(?=[A-Z])', review)
#             for segment in sentence_segments:
#                 if segment.strip():  # Ignore empty segments
#                     try:
#                         segment_text = re.split(r'\W+', segment.lower())
#                         segment_bow = dictionary.doc2bow(segment_text)
#                         segment_topic_dist = lda_model[segment_bow]
#                         segment_topic = max(segment_topic_dist, key=lambda x: x[1])[0]
#                         segment_topics.append((segment.strip(), segment_topic))
#                     except Exception as e:
#                         pass
#                         # print(f"Error processing segment: {segment.strip()}")
#                         # print(f"Error message: {e}")

#         return segment_topics

#     def map_segments_to_metrics(self, segment_sentiments):
#         metric_embeddings = self.similarity_model.encode(self.metrics, convert_to_tensor=True)
#         segment_texts = [segment for segment, _, _ in segment_sentiments]
#         segment_embeddings = self.similarity_model.encode(segment_texts, convert_to_tensor=True)

#         similarity_scores = util.pytorch_cos_sim(segment_embeddings, metric_embeddings)

#         segment_to_metric = {}
#         for i, segment in enumerate(segment_sentiments):
#             segment_text, sentiment_label, sentiment_score = segment
#             most_similar_metric_idx = torch.argmax(similarity_scores[i])  # Use GPU for argmax
#             most_similar_metric = self.metrics[most_similar_metric_idx.item()]  # Convert to Python int
#             segment_to_metric[segment_text] = (most_similar_metric, sentiment_label, sentiment_score)

#         return segment_to_metric

#     def analyze_reviews(self):
#         self.preprocess()
#         topics = self.get_topics()
#         # print("Topics and their top words:", topics)

#         segment_sentiments = self.analyze_sentiments()
#         # print("Segment sentiments and their topics:", segment_sentiments)

#         segment_to_metric = self.map_segments_to_metrics(segment_sentiments)
#         # print("Segment to metric mapping:", segment_to_metric)

#         # Convert to DataFrame
#         df_data = []
#         for review, glassdoor_id in zip(self.reviews, self.df['index']):
#             sentence_segments = re.split(r'(?<=\.)\s+(?=[A-Z])', review)
#             for segment in sentence_segments:
#                 if segment.strip():  # Ignore empty segments
#                     if segment.strip() in segment_to_metric:
#                         metric, sentiment_label, sentiment_score = segment_to_metric[segment.strip()]
#                         df_data.append([review, segment.strip(), metric, sentiment_label, sentiment_score, glassdoor_id])
#                     else:
#                         pass
#                         # print(f"Segment not found in mapping: {segment.strip()}")

#         df_reviews = pd.DataFrame(df_data, columns=['Review', 'Segment', 'Metric', 'Sentiment Label', 'Sentiment Score', 'index'])

#         # Calculate average sentiment score for each metric
#         metric_summary = df_reviews.groupby('Metric')['Sentiment Score'].mean().reset_index()

#         return df_reviews, metric_summary

# # Example usage:
# # analyzer = ReviewAnalyzer(df, metrics)
# # df_reviews, metric_summary = analyzer.analyze_reviews()


In [14]:
# # new and improved

# import pandas as pd
# from cuml.feature_extraction.text import TfidfVectorizer
# from transformers import pipeline
# from sentence_transformers import SentenceTransformer, util
# import cupy as cp
# import numpy as np
# import re
# import torch
# from tqdm import tqdm
# from gensim.models.ldamulticore import LdaMulticore
# from gensim.corpora import Dictionary

# class ReviewAnalyzer:
#     def __init__(self, df, metrics, subset_indices=None, model_name='cardiffnlp/twitter-roberta-base-sentiment'):
#         if subset_indices is not None:
#             self.df = df.iloc[subset_indices]
#         else:
#             self.df = df

#         self.reviews = self.df['Combined_Reviews']
#         self.metrics = metrics
#         self.vectorizer = TfidfVectorizer(stop_words='english')

#         # Disable parallelism before tokenization
#         os.environ["TOKENIZERS_PARALLELISM"] = "false"
#         self.sentiment_pipeline = pipeline("sentiment-analysis", model=model_name, device=0, batch_size=128)
#         self.similarity_model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2', device='cuda')
#         # Enable parallelism after tokenization
#         os.environ["TOKENIZERS_PARALLELISM"] = "true"
#         self.model_name = model_name

#     def preprocess(self):
#         self.X = self.vectorizer.fit_transform(self.reviews)
#         self.X = cp.array(self.X.toarray(), dtype=cp.float32)  # Convert to dense CuPy array

#     def get_topics(self):
#         texts = [re.split(r'\W+', review.lower()) for review in self.reviews]
#         dictionary = Dictionary(texts)
#         corpus = [dictionary.doc2bow(text) for text in texts]

#         # Initialize and train the Gensim LDA model with multicore support
#         lda_model = LdaMulticore(corpus, num_topics=len(self.metrics), id2word=dictionary, passes=10, workers=4, random_state=42)

#         topics = {}
#         for idx, topic in lda_model.print_topics(-1):
#             topics[idx] = topic

#         return topics

#     def analyze_sentiments(self):
#         segment_sentiments = []
#         batch_size = 128  # Adjust batch size based on your GPU memory
#         for i in tqdm(range(0, len(self.reviews), batch_size), desc="Processing Reviews for Sentiment Analysis"):
#             batch_reviews = self.reviews[i:i+batch_size]
#             batch_segments = []
#             for review in batch_reviews:
#                 sentence_segments = re.split(r'(?<=\.)\s+(?=[A-Z])', review)
#                 batch_segments.extend([segment.strip() for segment in sentence_segments if segment.strip()])

#             if batch_segments:
#                 try:
#                     sentiments = self.sentiment_pipeline(batch_segments)
#                     for segment, sentiment in zip(batch_segments, sentiments):
#                         sentiment_score = -sentiment['score'] if sentiment['label'] == 'LABEL_0' else sentiment['score']
#                         segment_sentiments.append((segment, sentiment['label'], sentiment_score))
#                 except Exception as e:
#                     pass
#                     # Handle exceptions if needed
#         return segment_sentiments

#     def analyze_topics(self):
#         segment_topics = []
#         texts = [re.split(r'\W+', review.lower()) for review in self.reviews]
#         dictionary = Dictionary(texts)
#         corpus = [dictionary.doc2bow(text) for text in texts]

#         # Initialize and train the Gensim LDA model with multicore support
#         lda_model = LdaMulticore(corpus, num_topics=len(self.metrics), id2word=dictionary, passes=10, workers=4, random_state=42)

#         for review in tqdm(self.reviews, desc="Processing Reviews for Topic Modeling"):
#             sentence_segments = re.split(r'(?<=\.)\s+(?=[A-Z])', review)
#             for segment in sentence_segments:
#                 if segment.strip():
#                     try:
#                         segment_text = re.split(r'\W+', segment.lower())
#                         segment_bow = dictionary.doc2bow(segment_text)
#                         segment_topic_dist = lda_model[segment_bow]
#                         segment_topic = max(segment_topic_dist, key=lambda x: x[1])[0]
#                         segment_topics.append((segment.strip(), segment_topic))
#                     except Exception as e:
#                         pass
#                         # Handle exceptions if needed
#         return segment_topics

#     def map_segments_to_metrics(self, segment_sentiments):
#         metric_embeddings = self.similarity_model.encode(self.metrics, convert_to_tensor=True)
#         segment_texts = [segment for segment, _, _ in segment_sentiments]
#         segment_embeddings = self.similarity_model.encode(segment_texts, convert_to_tensor=True)

#         similarity_scores = util.pytorch_cos_sim(segment_embeddings, metric_embeddings)

#         segment_to_metric = {}
#         for i, segment in enumerate(segment_sentiments):
#             segment_text, sentiment_label, sentiment_score = segment
#             most_similar_metric_idx = torch.argmax(similarity_scores[i])
#             most_similar_metric = self.metrics[most_similar_metric_idx.item()]
#             segment_to_metric[segment_text] = (most_similar_metric, sentiment_label, sentiment_score)

#         return segment_to_metric

#     def analyze_reviews(self):
#         self.preprocess()
#         topics = self.get_topics()

#         segment_sentiments = self.analyze_sentiments()

#         segment_to_metric = self.map_segments_to_metrics(segment_sentiments)

#         df_data = []
#         for review, glassdoor_id in zip(self.reviews, self.df['index']):
#             sentence_segments = re.split(r'(?<=\.)\s+(?=[A-Z])', review)
#             for segment in sentence_segments:
#                 if segment.strip():
#                     if segment.strip() in segment_to_metric:
#                         metric, sentiment_label, sentiment_score = segment_to_metric[segment.strip()]
#                         df_data.append([review, segment.strip(), metric, sentiment_label, sentiment_score, glassdoor_id])
#                     else:
#                         pass

#         df_reviews = pd.DataFrame(df_data, columns=['Review', 'Segment', 'Metric', 'Sentiment Label', 'Sentiment Score', 'index'])

#         metric_summary = df_reviews.groupby('Metric')['Sentiment Score'].mean().reset_index()

#         return df_reviews, metric_summary

# # Example usage:
# # analyzer = ReviewAnalyzer(df, metrics)
# # df_reviews, metric_summary = analyzer.analyze_reviews()

In [15]:
# # single initialization

# import pandas as pd
# from cuml.feature_extraction.text import TfidfVectorizer
# from transformers import pipeline
# from sentence_transformers import SentenceTransformer, util
# import cupy as cp
# import numpy as np
# import re
# import torch
# from tqdm import tqdm
# from gensim.models.ldamulticore import LdaMulticore
# from gensim.corpora import Dictionary
# import os

# class ReviewAnalyzer:
#     def __init__(self, df, metrics, subset_indices=None, model_name='cardiffnlp/twitter-roberta-base-sentiment'):
#         if subset_indices is not None:
#             self.df = df.iloc[subset_indices]
#         else:
#             self.df = df

#         self.reviews = self.df['Combined_Reviews']
#         self.metrics = metrics
#         self.vectorizer = TfidfVectorizer(stop_words='english')

#         # Disable parallelism before tokenization
#         os.environ["TOKENIZERS_PARALLELISM"] = "false"
#         self.sentiment_pipeline = pipeline("sentiment-analysis", model=model_name, device=0, batch_size=128)
#         self.similarity_model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2', device='cuda')
#         # Enable parallelism after tokenization
#         os.environ["TOKENIZERS_PARALLELISM"] = "true"
#         self.model_name = model_name

#     def preprocess(self):
#         self.X = self.vectorizer.fit_transform(self.reviews)
#         self.X = cp.array(self.X.toarray(), dtype=cp.float32)  # Convert to dense CuPy array

#     def get_topics(self):
#         texts = [re.split(r'\W+', review.lower()) for review in self.reviews]
#         dictionary = Dictionary(texts)
#         corpus = [dictionary.doc2bow(text) for text in texts]

#         # Initialize and train the Gensim LDA model with multicore support
#         lda_model = LdaMulticore(corpus, num_topics=len(self.metrics), id2word=dictionary, passes=10, workers=4, random_state=42)

#         topics = {}
#         for idx, topic in lda_model.print_topics(-1):
#             topics[idx] = topic

#         return topics

#     def analyze_sentiments(self):
#         segment_sentiments = []
#         batch_size = 128  # Adjust batch size based on your GPU memory
#         for i in tqdm(range(0, len(self.reviews), batch_size), desc="Processing Reviews for Sentiment Analysis"):
#             batch_reviews = self.reviews[i:i+batch_size]
#             batch_segments = []
#             for review in batch_reviews:
#                 sentence_segments = re.split(r'(?<=\.)\s+(?=[A-Z])', review)
#                 batch_segments.extend([segment.strip() for segment in sentence_segments if segment.strip()])

#             if batch_segments:
#                 try:
#                     sentiments = self.sentiment_pipeline(batch_segments)
#                     for segment, sentiment in zip(batch_segments, sentiments):
#                         sentiment_score = -sentiment['score'] if sentiment['label'] == 'LABEL_0' else sentiment['score']
#                         segment_sentiments.append((segment, sentiment['label'], sentiment_score))
#                 except Exception as e:
#                     pass
#                     # Handle exceptions if needed
#         return segment_sentiments

#     def analyze_topics(self):
#         segment_topics = []
#         texts = [re.split(r'\W+', review.lower()) for review in self.reviews]
#         dictionary = Dictionary(texts)
#         corpus = [dictionary.doc2bow(text) for text in texts]

#         # Initialize and train the Gensim LDA model with multicore support
#         lda_model = LdaMulticore(corpus, num_topics=len(self.metrics), id2word=dictionary, passes=10, workers=4, random_state=42)

#         for review in tqdm(self.reviews, desc="Processing Reviews for Topic Modeling"):
#             sentence_segments = re.split(r'(?<=\.)\s+(?=[A-Z])', review)
#             for segment in sentence_segments:
#                 if segment.strip():
#                     try:
#                         segment_text = re.split(r'\W+', segment.lower())
#                         segment_bow = dictionary.doc2bow(segment_text)
#                         segment_topic_dist = lda_model[segment_bow]
#                         segment_topic = max(segment_topic_dist, key=lambda x: x[1])[0]
#                         segment_topics.append((segment.strip(), segment_topic))
#                     except Exception as e:
#                         pass
#                         # Handle exceptions if needed
#         return segment_topics

#     def map_segments_to_metrics(self, segment_sentiments):
#         metric_embeddings = self.similarity_model.encode(self.metrics, convert_to_tensor=True)
#         segment_texts = [segment for segment, _, _ in segment_sentiments]
#         segment_embeddings = self.similarity_model.encode(segment_texts, convert_to_tensor=True)

#         similarity_scores = util.pytorch_cos_sim(segment_embeddings, metric_embeddings)

#         segment_to_metric = {}
#         for i, segment in enumerate(segment_sentiments):
#             segment_text, sentiment_label, sentiment_score = segment
#             most_similar_metric_idx = torch.argmax(similarity_scores[i])
#             most_similar_metric = self.metrics[most_similar_metric_idx.item()]
#             segment_to_metric[segment_text] = (most_similar_metric, sentiment_label, sentiment_score)

#         return segment_to_metric

#     def analyze_reviews(self):
#         self.preprocess()
#         topics = self.get_topics()

#         segment_sentiments = self.analyze_sentiments()

#         segment_to_metric = self.map_segments_to_metrics(segment_sentiments)

#         df_data = []
#         for review, glassdoor_id in zip(self.reviews, self.df['index']):
#             sentence_segments = re.split(r'(?<=\.)\s+(?=[A-Z])', review)
#             for segment in sentence_segments:
#                 if segment.strip():
#                     if segment.strip() in segment_to_metric:
#                         metric, sentiment_label, sentiment_score = segment_to_metric[segment.strip()]
#                         df_data.append([review, segment.strip(), metric, sentiment_label, sentiment_score, glassdoor_id])
#                     else:
#                         pass

#         df_reviews = pd.DataFrame(df_data, columns=['Review', 'Segment', 'Metric', 'Sentiment Label', 'Sentiment Score', 'index'])

#         metric_summary = df_reviews.groupby('Metric')['Sentiment Score'].mean().reset_index()

#         return df_reviews, metric_summary

# # Example usage:
# # analyzer = ReviewAnalyzer(df, metrics)
# # df_reviews, metric_summary = analyzer.analyze_reviews()

In [16]:
# # single initialization

# import pandas as pd
# from cuml.feature_extraction.text import TfidfVectorizer
# from transformers import pipeline
# from sentence_transformers import SentenceTransformer, util
# import cupy as cp
# import numpy as np
# import re
# import torch
# from tqdm import tqdm
# import os
# from sklearn.decomposition import LatentDirichletAllocation
# from sklearn.feature_extraction.text import CountVectorizer

# class ReviewAnalyzer:
#     def __init__(self, df, metrics, subset_indices=None, model_name='cardiffnlp/twitter-roberta-base-sentiment'):
#         if subset_indices is not None:
#             self.df = df.iloc[subset_indices]
#         else:
#             self.df = df

#         self.reviews = self.df['Combined_Reviews']
#         self.metrics = metrics
#         self.vectorizer = TfidfVectorizer(stop_words='english')

#         # Disable parallelism before tokenization
#         os.environ["TOKENIZERS_PARALLELISM"] = "false"
#         self.sentiment_pipeline = pipeline("sentiment-analysis", model=model_name, device=0, batch_size=128)
#         self.similarity_model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2', device='cuda')
#         # Enable parallelism after tokenization
#         os.environ["TOKENIZERS_PARALLELISM"] = "true"
#         self.model_name = model_name

#     def preprocess(self):
#         self.X = self.vectorizer.fit_transform(self.reviews)
#         self.X = cp.array(self.X.toarray(), dtype=cp.float32)  # Convert to dense CuPy array

#     def get_topics(self):
#         texts = [re.split(r'\W+', review.lower()) for review in self.reviews]
#         texts = [' '.join(text) for text in texts]
#         vectorizer = CountVectorizer(stop_words='english')
#         X = vectorizer.fit_transform(texts)

#         lda_model = LatentDirichletAllocation(n_components=len(self.metrics), random_state=42, n_jobs=-1)
#         lda_model.fit(X)

#         topics = {}
#         for idx, topic in enumerate(lda_model.components_):
#             topics[idx] = ' '.join([vectorizer.get_feature_names_out()[i] for i in topic.argsort()[:-11:-1]])

#         return topics

#     def analyze_sentiments(self):
#         segment_sentiments = []
#         batch_size = 128  # Adjust batch size based on your GPU memory
#         for i in tqdm(range(0, len(self.reviews), batch_size), desc="Processing Reviews for Sentiment Analysis"):
#             batch_reviews = self.reviews[i:i+batch_size]
#             batch_segments = []
#             for review in batch_reviews:
#                 sentence_segments = re.split(r'(?<=\.)\s+(?=[A-Z])', review)
#                 batch_segments.extend([segment.strip() for segment in sentence_segments if segment.strip()])

#             if batch_segments:
#                 try:
#                     sentiments = self.sentiment_pipeline(batch_segments)
#                     for segment, sentiment in zip(batch_segments, sentiments):
#                         sentiment_score = -sentiment['score'] if sentiment['label'] == 'LABEL_0' else sentiment['score']
#                         segment_sentiments.append((segment, sentiment['label'], sentiment_score))
#                 except Exception as e:
#                     pass
#                     # Handle exceptions if needed
#         return segment_sentiments

#     def analyze_topics(self):
#         segment_topics = []
#         texts = [re.split(r'\W+', review.lower()) for review in self.reviews]
#         texts = [' '.join(text) for text in texts]
#         vectorizer = CountVectorizer(stop_words='english')
#         X = vectorizer.fit_transform(texts)

#         lda_model = LatentDirichletAllocation(n_components=len(self.metrics), random_state=42, n_jobs=-1)
#         lda_model.fit(X)

#         for review in tqdm(self.reviews, desc="Processing Reviews for Topic Modeling"):
#             sentence_segments = re.split(r'(?<=\.)\s+(?=[A-Z])', review)
#             for segment in sentence_segments:
#                 if segment.strip():
#                     try:
#                         segment_text = ' '.join(re.split(r'\W+', segment.lower()))
#                         segment_vector = vectorizer.transform([segment_text])
#                         segment_topic_dist = lda_model.transform(segment_vector)
#                         segment_topic = segment_topic_dist.argmax()
#                         segment_topics.append((segment.strip(), segment_topic))
#                     except Exception as e:
#                         pass
#                         # Handle exceptions if needed
#         return segment_topics

#     def map_segments_to_metrics(self, segment_sentiments):
#         metric_embeddings = self.similarity_model.encode(self.metrics, convert_to_tensor=True)
#         segment_texts = [segment for segment, _, _ in segment_sentiments]
#         segment_embeddings = self.similarity_model.encode(segment_texts, convert_to_tensor=True)

#         similarity_scores = util.pytorch_cos_sim(segment_embeddings, metric_embeddings)

#         segment_to_metric = {}
#         for i, segment in enumerate(segment_sentiments):
#             segment_text, sentiment_label, sentiment_score = segment
#             most_similar_metric_idx = torch.argmax(similarity_scores[i])
#             most_similar_metric = self.metrics[most_similar_metric_idx.item()]
#             segment_to_metric[segment_text] = (most_similar_metric, sentiment_label, sentiment_score)

#         return segment_to_metric

#     def analyze_reviews(self):
#         self.preprocess()
#         topics = self.get_topics()

#         segment_sentiments = self.analyze_sentiments()

#         segment_to_metric = self.map_segments_to_metrics(segment_sentiments)

#         df_data = []
#         for review, glassdoor_id in zip(self.reviews, self.df['index']):
#             sentence_segments = re.split(r'(?<=\.)\s+(?=[A-Z])', review)
#             for segment in sentence_segments:
#                 if segment.strip():
#                     if segment.strip() in segment_to_metric:
#                         metric, sentiment_label, sentiment_score = segment_to_metric[segment.strip()]
#                         df_data.append([review, segment.strip(), metric, sentiment_label, sentiment_score, glassdoor_id])
#                     else:
#                         pass

#         df_reviews = pd.DataFrame(df_data, columns=['Review', 'Segment', 'Metric', 'Sentiment Label', 'Sentiment Score', 'index'])

#         metric_summary = df_reviews.groupby('Metric')['Sentiment Score'].mean().reset_index()

#         return df_reviews, metric_summary

# # Example usage:
# # analyzer = ReviewAnalyzer(df, metrics)
# # df_reviews, metric_summary = analyzer.analyze_reviews()

#### In-use

In [17]:
# run on either cpu or gpu or tpu

import pandas as pd
from transformers import pipeline
from sentence_transformers import SentenceTransformer, util
import numpy as np
import re
import torch
from tqdm import tqdm
import os
from sklearn.decomposition import LatentDirichletAllocation
from sklearn.feature_extraction.text import CountVectorizer
import tensorflow as tf

# Try to import GPU-specific libraries
try:
    import cupy as cp
    from cuml.feature_extraction.text import TfidfVectorizer as CuTfidfVectorizer
    gpu_available = True
except ImportError:
    from sklearn.feature_extraction.text import TfidfVectorizer as SkTfidfVectorizer
    gpu_available = False

def is_cuda_available():
    return torch.cuda.is_available()

def is_tpu_available():
    try:
        tpu = tf.distribute.cluster_resolver.TPUClusterResolver()
        return True
    except ValueError:
        return False

class ReviewAnalyzer:
    def __init__(self, df, metrics, subset_indices=None, model_name='cardiffnlp/twitter-roberta-base-sentiment'):
        if subset_indices is not None:
            self.df = df.iloc[subset_indices]
        else:
            self.df = df

        self.reviews = self.df['Combined_Reviews']
        self.metrics = metrics

        # Choose the appropriate vectorizer based on GPU availability
        if gpu_available and is_cuda_available():
            self.vectorizer = CuTfidfVectorizer(stop_words='english')
        else:
            self.vectorizer = SkTfidfVectorizer(stop_words='english')

        # Disable parallelism before tokenization
        os.environ["TOKENIZERS_PARALLELISM"] = "false"

        # Check for TPU availability
        if is_tpu_available():
            resolver = tf.distribute.cluster_resolver.TPUClusterResolver()
            tf.config.experimental_connect_to_cluster(resolver)
            tf.tpu.experimental.initialize_tpu_system(resolver)
            strategy = tf.distribute.TPUStrategy(resolver)
            device = 'tpu'
            self.similarity_model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2', device='cpu')
        elif is_cuda_available():
            device = 0
            self.similarity_model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2', device='cuda')
        else:
            device = -1
            self.similarity_model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2', device='cpu')

        self.sentiment_pipeline = pipeline("sentiment-analysis", model=model_name, device=device, batch_size=128)

        # Enable parallelism after tokenization
        os.environ["TOKENIZERS_PARALLELISM"] = "true"
        self.model_name = model_name

    def preprocess(self):
        self.X = self.vectorizer.fit_transform(self.reviews)
        if gpu_available and is_cuda_available():
            self.X = cp.array(self.X.toarray(), dtype=cp.float32)  # Convert to dense CuPy array
        else:
            self.X = np.array(self.X.toarray(), dtype=np.float32)  # Convert to dense NumPy array

    def get_topics(self):
        texts = [re.split(r'\W+', review.lower()) for review in self.reviews]
        texts = [' '.join(text) for text in texts]
        vectorizer = CountVectorizer(stop_words='english')
        X = vectorizer.fit_transform(texts)

        lda_model = LatentDirichletAllocation(n_components=len(self.metrics), random_state=42, n_jobs=-1)
        lda_model.fit(X)

        topics = {}
        for idx, topic in enumerate(lda_model.components_):
            topics[idx] = ' '.join([vectorizer.get_feature_names_out()[i] for i in topic.argsort()[:-11:-1]])

        return topics

    def analyze_sentiments(self):
        segment_sentiments = []
        batch_size = 128  # Adjust batch size based on your GPU memory
        for i in tqdm(range(0, len(self.reviews), batch_size), desc="Processing Reviews for Sentiment Analysis"):
            batch_reviews = self.reviews[i:i+batch_size]
            batch_segments = []
            for review in batch_reviews:
                sentence_segments = re.split(r'(?<=\.)\s+(?=[A-Z])', review)
                batch_segments.extend([segment.strip() for segment in sentence_segments if segment.strip()])

            if batch_segments:
                try:
                    sentiments = self.sentiment_pipeline(batch_segments)
                    for segment, sentiment in zip(batch_segments, sentiments):
                        sentiment_score = -sentiment['score'] if sentiment['label'] == 'LABEL_0' else sentiment['score']
                        segment_sentiments.append((segment, sentiment['label'], sentiment_score))
                except Exception as e:
                    pass
                    # Handle exceptions if needed
        return segment_sentiments

    # def analyze_sentiments(self):
    #     segment_sentiments = []
    #     batch_size = 128  # Adjust batch size based on your GPU memory
    #     for i in tqdm(range(0, len(self.reviews), batch_size), desc="Processing Reviews for Sentiment Analysis"):
    #         batch_reviews = self.reviews[i:i+batch_size]
    #         batch_segments = []
    #         for review in batch_reviews:
    #             sentence_segments = re.split(r'(?<=\.)\s+(?=[A-Z])', review)
    #             batch_segments.extend([segment.strip() for segment in sentence_segments if segment.strip()])

    #         if batch_segments:
    #             try:
    #                 sentiments = self.sentiment_pipeline(batch_segments)
    #                 for segment, sentiment in zip(batch_segments, sentiments):
    #                     sentiment_score = -sentiment['score'] if sentiment['label'] == 'LABEL_0' else sentiment['score']
    #                     segment_sentiments.append((segment, sentiment['label'], sentiment_score))
    #             except Exception as e:
    #                 print(f"Error processing batch: {e}")
    #                 # Handle exceptions if needed
    #     return segment_sentiments


    def analyze_topics(self):
        segment_topics = []
        texts = [re.split(r'\W+', review.lower()) for review in self.reviews]
        texts = [' '.join(text) for text in texts]
        vectorizer = CountVectorizer(stop_words='english')
        X = vectorizer.fit_transform(texts)

        lda_model = LatentDirichletAllocation(n_components=len(self.metrics), random_state=42, n_jobs=-1)
        lda_model.fit(X)

        for review in tqdm(self.reviews, desc="Processing Reviews for Topic Modeling"):
            sentence_segments = re.split(r'(?<=\.)\s+(?=[A-Z])', review)
            for segment in sentence_segments:
                if segment.strip():
                    try:
                        segment_text = ' '.join(re.split(r'\W+', segment.lower()))
                        segment_vector = vectorizer.transform([segment_text])
                        segment_topic_dist = lda_model.transform(segment_vector)
                        segment_topic = segment_topic_dist.argmax()
                        segment_topics.append((segment.strip(), segment_topic))
                    except Exception as e:
                        pass
                        # Handle exceptions if needed
        return segment_topics

    def map_segments_to_metrics(self, segment_sentiments):
        metric_embeddings = self.similarity_model.encode(self.metrics, convert_to_tensor=True)
        segment_texts = [segment for segment, _, _ in segment_sentiments]
        segment_embeddings = self.similarity_model.encode(segment_texts, convert_to_tensor=True)

        similarity_scores = util.pytorch_cos_sim(segment_embeddings, metric_embeddings)

        segment_to_metric = {}
        for i, segment in enumerate(segment_sentiments):
            segment_text, sentiment_label, sentiment_score = segment
            most_similar_metric_idx = torch.argmax(similarity_scores[i])
            most_similar_metric = self.metrics[most_similar_metric_idx.item()]
            segment_to_metric[segment_text] = (most_similar_metric, sentiment_label, sentiment_score)

        return segment_to_metric

    def analyze_reviews(self):
        self.preprocess()
        topics = self.get_topics()

        segment_sentiments = self.analyze_sentiments()

        segment_to_metric = self.map_segments_to_metrics(segment_sentiments)

        df_data = []
        for review, glassdoor_id in zip(self.reviews, self.df['index']):
            sentence_segments = re.split(r'(?<=\.)\s+(?=[A-Z])', review)
            for segment in sentence_segments:
                if segment.strip():
                    if segment.strip() in segment_to_metric:
                        metric, sentiment_label, sentiment_score = segment_to_metric[segment.strip()]
                        df_data.append([review, segment.strip(), metric, sentiment_label, sentiment_score, glassdoor_id])
                    else:
                        pass

        df_reviews = pd.DataFrame(df_data, columns=['Review', 'Segment', 'Metric', 'Sentiment Label', 'Sentiment Score', 'index'])

        metric_summary = df_reviews.groupby('Metric')['Sentiment Score'].mean().reset_index()

        return df_reviews, metric_summary

# Example usage:
# analyzer = ReviewAnalyzer(df, metrics)
# df_reviews, metric_summary = analyzer.analyze_reviews()

### Unzip Dataset

In [18]:
import zipfile
import time
import os

if os.path.isfile("text_reviews.json"):
  print("File is already unzipped.")
else:
  # Specify the path to your .zip file and the extraction directory
  zip_file_path = '/content/drive/MyDrive/Colab Notebooks/FINA4359/text_reviews.zip'
  extract_to_path = ''

  # Open and extract the .zip file
  with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
      start = time.time()
      zip_ref.extractall(extract_to_path)
      print(time.time() - start)

24.530701875686646


### Load Dataset

In [19]:
## Load real data

import json
import pandas as pd

try:
    df = pd.read_json("text_reviews.json")         ## load in dataset
    df.fillna("", inplace=True)

    # df1 = pd.read_json("text_reviews1.json")         ## load in dataset
    # df1.fillna("", inplace=True)

    # df_c = pd.concat([df, df1], ignore_index=True)
    # df = df_c
    # df.head()

except ValueError as e:
    print(f"Error reading JSON file: {e}")

# Combine review columns into one paragraph per group
df['Combined_Reviews'] = df[['summary', 'pros', 'cons', 'advice']].agg(' '.join, axis=1)

# Drop the original review columns if needed
df = df.drop(columns=['summary', 'pros', 'cons', 'advice'])

print(f"rows: {len(df)}")

rows: 3431066


In [ ]:
# df.head()

In [ ]:
## Set date of data transformation
from datetime import date
str(date.today())

In [ ]:
from datetime import datetime

# Get the current time
now = datetime.now()

# Format the time
current_time = now.strftime("%H:%M:%S")
print(current_time)

In [ ]:
import os

# Define the folder path
folder_path = f"/content/drive/MyDrive/Colab Notebooks/FINA4359/output/{date.today()}"

# Check if the folder exists
if not os.path.exists(folder_path):
    # Create the folder
    os.makedirs(folder_path)
    print(f"Folder '{folder_path}' created.")
else:
    print(f"Folder '{folder_path}' already exists.")


### Initiate Run

In [ ]:
###### Example usage
# reviews = [
#     "Working at XYZ Corp has been a fantastic experience. The team is supportive, and the management genuinely cares about employee growth. I’ve learned so much and will always cherish my time here.",
#     "My time at ABC Inc. was a mixed bag. While the projects were interesting and challenging, I felt that communication from upper management could have been better. Overall, it was a valuable learning experience.",
#     "I’ve enjoyed every moment at DEF Ltd. The company culture is inclusive, and there are ample opportunities for professional development. I’m grateful for the mentorship and support I received.",
#     "GHI Co. has a lot of potential, but there are areas for improvement. The workload can be overwhelming at times, and clearer processes would help. Despite this, I appreciated the collaborative environment.",
#     "JKL Enterprises is a great place to work. The benefits are excellent, and the work-life balance is respected. I’m leaving with a wealth of knowledge and many fond memories.",
#     "My experience at MNO Group was generally positive, but there were some challenges. The pace was fast, and sometimes it felt like there wasn’t enough support. However, the team spirit was strong, and I learned a lot.",
#     "Working at PQR Solutions has been a rewarding journey. The innovative projects and the supportive team made every day exciting. I’m grateful for the opportunities to grow and develop my skills.",
#     "STU Technologies provided a dynamic work environment. While the fast-paced nature was challenging, it pushed me to improve and adapt quickly. The leadership was inspiring, and I leave with a sense of accomplishment.",
#     "VWX Innovations was a place of constant learning. The collaborative culture and the emphasis on creativity made it a great place to work. I appreciate the guidance and the friendships I’ve made here.",
#     "YZA Enterprises had its ups and downs. The workload was heavy, but the team’s camaraderie made it manageable. I’m thankful for the experiences and the professional growth I achieved."
# ]

import gc

# Analyze the reviews and print the results (this will simulate the process)
u = 257
v = 258

# loop timer
big_start = time.time()

df = df.iloc[list(range(u*10000, v*10000))]
df.reset_index(drop=True, inplace=True)

for c in range(u-u, v-u):
    ## Real usage
    a = 0 + (c*10000)
    b = 10000 + (c*10000)
    # a = 0
    # b = 10000
    print("="*100)
    print(f"Run {c}: row {a+u*10000} to {b+u*10000}")
    subset_indices = list(range(a, b))


    metrics = [
    "Employee Satisfaction",
    "Communication Effectiveness",
    "Workload Management",
    "Professional Development",
    "Work-Life Balance",
    "Team Collaboration",
    "Leadership Quality",
    "Innovation Encouragement",
    "Career Advancement Opportunities",
    "Employee Recognition",
    "Management Quality",
    "Benefits and Compensation",
    "Company Culture",
    "Job Security",
    "Productivity",
    "Quality of Work",
    "Credibility",
    "Leadership"
    ]

    analyzer = ReviewAnalyzer(df, metrics, subset_indices)

    try:
        start = time.time()

        df_reviews, metric_summary = analyzer.analyze_reviews()
        # print(df_reviews.head())
        # print(metric_summary.head())

        end = time.time()

        print(f" Running Time: {end - start}")
    # except MemoryError as e:
    except Exception as e:
        print(e)
        try:
            print("something went wrong")
            df_reviews.to_parquet('output/output_abr.parquet', engine='pyarrow', index=False)
        except Exception as e:
            print(e)

    ## save output as
    # df_reviews.to_parquet(f"output/{date.today()}/{u+c}_{current_time}.parquet", engine='pyarrow', index=False)
    df_reviews.to_parquet(f"/content/drive/MyDrive/Colab Notebooks/FINA4359/output/{date.today()}/{u+c}_{current_time}.parquet", engine='pyarrow', index=False)

print("=============================")
print(f"   Big timer: {time.time() - big_start}  ")
print("=============================")


In [ ]:
# ## sanity check: Read the Parquet file
# df_loaded = pd.read_parquet('reviews.parquet', engine='pyarrow')
# df_loaded

In [ ]:
metric_summary

In [ ]:
cp.get_default_memory_pool().free_all_blocks()

In [ ]:
df

In [ ]:
df_reviews

In [ ]:
df_reviews.to_parquet(f"/content/drive/MyDrive/Colab Notebooks/FINA4359/output/{date.today()}/{u+c}_{current_time}.parquet", engine='pyarrow', index=False)